In [5]:
import collections
import math
import numpy as np
import pickle
import random
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
import os
import pandas as pd
from torch.utils.data import DataLoader, Dataset
from sklearn.model_selection import train_test_split
from tqdm import tqdm

In [6]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using", device)

Using cuda


In [7]:
folder_path = "C:/Users/Kieran/Documents/Master Thesis Data/Datasets/MouseCentric"

dataframes = []

for filename in os.listdir(folder_path):
    if filename.endswith(".csv"):
        file_path = os.path.join(folder_path, filename)
        df = pd.read_csv(file_path)
        dataframes.append(df)
        
data = pd.concat(dataframes, ignore_index=True)
data = data.drop(columns = ['frame_number'])
data

,"('nose', 'x')","('nose', 'y')","('nose', 'likelihood')","('H1R', 'x')","('H1R', 'y')","('H1R', 'likelihood')","('H2R', 'x')","('H2R', 'y')","('H2R', 'likelihood')","('H1L', 'x')",...,"('tail', 'x')","('tail', 'y')","('tail', 'likelihood')","('S2', 'x')","('S2', 'y')","('S2', 'likelihood')","('S1', 'x')","('S1', 'y')","('S1', 'likelihood')",mouse_no
0,97.501312,72.446838,0.999969,76.267792,76.465271,0.999760,54.404541,76.934204,0.996968,99.304657,...,-94.751236,-99.595276,0.998689,-45.061890,-37.560608,0.994912,292.150818,318.674744,0.995648,11.4
1,94.315247,71.871887,0.999950,73.173645,75.850555,0.999568,55.222382,78.693970,0.997916,96.209991,...,-96.711456,-101.978439,0.999059,-48.097229,-40.080200,0.996069,295.781158,321.270813,0.993003,11.4
2,91.164520,73.615448,0.999945,70.528534,77.226471,0.999638,53.560516,77.271362,0.999153,93.589539,...,-99.503922,-102.734909,0.999419,-49.006653,-42.229187,0.986211,299.371765,323.419800,0.987800,11.4
3,95.102051,70.977386,0.999765,72.682373,74.588409,0.999534,54.546295,74.633301,0.999468,96.380615,...,-101.413986,-103.451233,0.999285,-49.794189,-44.825256,0.981083,303.137299,326.057861,0.994152,11.4
4,110.448181,70.909393,0.999855,91.858368,74.520416,0.999252,70.006622,74.565308,0.998945,108.407593,...,-103.237167,-98.591965,0.999156,-51.617371,-44.893250,0.983985,304.960480,326.125854,0.995609,11.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1102495,92.895691,-22.675415,0.999232,85.470825,-11.074768,0.999924,75.502136,-8.060303,0.999492,84.703430,...,-24.412476,116.388947,0.999563,-33.810913,56.276733,0.998793,525.421082,367.743622,0.998882,88.3
1102496,94.005859,-17.802216,0.999680,87.183533,-7.193573,0.999842,75.731262,-6.426147,0.999562,87.588257,...,-24.296509,116.438507,0.999529,-33.658234,56.326294,0.998716,525.653503,367.694061,0.998840,88.3
1102497,106.110474,-17.802216,0.999622,96.254883,-7.193573,0.999950,82.021301,-3.780365,0.999656,94.855774,...,-24.760468,115.177551,0.999215,-33.469727,56.024719,0.999035,525.961548,367.694061,0.998980,88.3
1102498,106.110474,-17.802216,0.999570,96.254883,-7.193573,0.999939,82.021301,-3.780365,0.999616,94.855774,...,-24.760468,114.962799,0.999225,-33.469727,55.950989,0.998865,525.961548,367.694061,0.998603,88.3


## Base model

In [8]:
dropout_prob   = 0.5
embedding_size = 32
epoch_num      = 10
hidden_size    = 16
layer_num      = 2
learning_rate  = 1e-3
seq_size       = 25 #25 frames is equal to 1 second of video, maybe use 50?
pred_window    = 1

In [17]:
class SeqDataset(Dataset):
    def __init__(self, device, seq_size, dataframe, pred_window):
        super(SeqDataset, self).__init__()
        self.device      = device
        self.seq_size    = seq_size
        self.dataframe   = dataframe
        self.pred_window = pred_window
        self.target_data = dataframe.filter(regex='x|y')

    def __len__(self):
        return len(self.dataframe) - self.seq_size - 1

    def __getitem__(self, idx):
        in_seq = torch.tensor(self.dataframe.iloc[idx:idx + self.seq_size].values, dtype=torch.float, device=self.device)
        target_seq = torch.tensor(self.target_data.iloc[idx + self.pred_window:idx + self.seq_size + self.pred_window].values, dtype=torch.float, device=self.device)
        return in_seq, target_seq

In [18]:
# Split the data into train, validation, and test using the full video sizes (so that videos are not split into different sets)
train_data, test_data = train_test_split(data, test_size= 11250*int(0.2*98), shuffle=False)
n_train_vids = len(train_data)/11250
train_data, val_data = train_test_split(train_data, test_size = 11250 * int(0.1 * n_train_vids), shuffle=False)

# Create SeqDataset instances for the train, validation, and test sets
train_dataset = SeqDataset(device, seq_size, train_data, pred_window)
val_dataset = SeqDataset(device, seq_size, val_data, pred_window)
test_dataset = SeqDataset(device, seq_size, test_data, pred_window)

# Create DataLoader instances for batching
batch_size = 64  # Adjust to your desired batch size
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, drop_last=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, drop_last=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, drop_last=True)

In [19]:
class Model(nn.Module):
    def __init__(self, dropout_prob, hidden_size, layer_num, input_size, output_size):
        super(Model, self).__init__()
#         self.embedding = nn.Embedding(vocab_size, embedding_size)
        self.gru       = nn.GRU(input_size, hidden_size, layer_num, batch_first=True, dropout=dropout_prob)
        self.linear    = nn.Linear(hidden_size, output_size)

    def forward(self, in_sequence, hidden_state=None):
#         embedding_seq            = self.embedding(in_sequence)
        hidden_seq, hidden_state = self.gru(in_sequence, hidden_state)
        out_seq                  = self.linear(hidden_seq)
        return out_seq, hidden_state
    
    def draw(self, in_sequence, logit_temp=1.0):
        out_seq, _  = self(in_sequence)
        prob_dist   = torch.softmax(out_seq[0, -1] / logit_temp, 0)
        rand_sample = torch.multinomial(prob_dist, 1).item()                   
        return rand_sample

In [22]:
input_size = next(iter(train_loader))[0].size(-1)
print("Input size:", input_size)
output_size = next(iter(train_loader))[1].size(-1)
print("Output size:", output_size)
model = Model(dropout_prob, hidden_size, layer_num, input_size, output_size).to(device)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), learning_rate)

Input size: 43
Output size: 28


In [25]:
min_loss = float("inf")

for epoch in range(epoch_num):
    
    model.train()
    train_loss = 0.0
    
    i = 0
    for in_seq, target_seq in tqdm(train_loader, desc=f"Epoch {epoch}/{epoch_num}"):
        
        out_seq, _  = model(in_seq)
        loss        = criterion(out_seq, target_seq)
        train_loss += loss.item()
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        i += 1

    train_loss       /= len(train_loader)
    # train_perplexity  = np.exp(train_loss)
    print(f"Train loss: {train_loss:.4f}")

    model.eval()
    val_loss = 0.0

    with torch.no_grad():
        for in_seq, target_seq in val_loader:
            out_seq, _  = model(in_seq)
            loss        = criterion(out_seq, target_seq)
            val_loss   += loss.item()

    val_loss       /= len(val_loader)
    # val_perplexity  = np.exp(val_loss)
    print(f"Val loss: {val_loss:.4f}")

    if val_loss < min_loss:
        min_loss = val_loss
        # torch.save(model.state_dict(), "model_pred_3.pt")

Epoch 0/10:   0%|                                                                    | 5/12655 [00:00<10:24, 20.26it/s]

torch.Size([64, 25, 43])
torch.Size([64, 25, 43])
torch.Size([64, 25, 43])
torch.Size([64, 25, 43])
torch.Size([64, 25, 43])
torch.Size([64, 25, 43])


Epoch 0/10:   0%|                                                                   | 11/12655 [00:00<09:32, 22.08it/s]

torch.Size([64, 25, 43])
torch.Size([64, 25, 43])
torch.Size([64, 25, 43])
torch.Size([64, 25, 43])
torch.Size([64, 25, 43])
torch.Size([64, 25, 43])


Epoch 0/10:   0%|                                                                   | 18/12655 [00:00<08:23, 25.11it/s]

...

Epoch 0/10:   0%|▎                                                                  | 60/12655 [00:02<07:41, 27.31it/s]

torch.Size([64, 25, 43])
torch.Size([64, 25, 43])
torch.Size([64, 25, 43])
torch.Size([64, 25, 43])
torch.Size([64, 25, 43])


Epoch 0/10:   1%|▎                                                                  | 64/12655 [00:02<07:58, 26.29it/s]

torch.Size([64, 25, 43])
torch.Size([64, 25, 43])
torch.Size([64, 25, 43])
Train loss: 72.3751


KeyboardInterrupt: 

In [64]:
model.load_state_dict(torch.load("model_pred_2.pt"))
model.eval

<bound method Module.eval of Model(
  (gru): GRU(43, 16, num_layers=2, batch_first=True, dropout=0.5)
  (linear): Linear(in_features=16, out_features=28, bias=True)
)>

In [65]:
test_loss = 0.0

for in_seq, target_seq in tqdm(test_loader):
    out_seq, _ = model(in_seq)
    loss = criterion(out_seq, target_seq)
    test_loss += loss.item()

#     x_coord = target_seq.cpu()[0, -1][::2]
#     y_coord = target_seq.cpu()[0, -1][1::2]
#     x_coord_pred = out_seq.cpu().detach().numpy()[0, -1][::2]
#     y_coord_pred = out_seq.cpu().detach().numpy()[0, -1][1::2]
    
    
#     plt.figure(figsize=(8,6))
#     plt.scatter(x_coord, y_coord, label = "Target")
#     plt.scatter(x_coord_pred, y_coord_pred, label = "Prediction")
#     plt.xlabel('X Coordinates')
#     plt.ylabel('Y Coordinates')
#     plt.title("Output test")
#     plt.legend()
#     plt.xlim(-800, 800)
#     plt.ylim(-650, 650)
#     plt.axhline(0, color='black', linewidth=0.8, linestyle='--')
#     plt.axvline(0, color='black', linewidth=0.8, linestyle='--')
#     plt.show()

test_loss /= len(test_loader)
print(f"Test loss: {test_loss:.4f}")

100%|██████████████████████████████████████████████████████████████████████████████| 3340/3340 [00:35<00:00, 94.74it/s]

Test loss: 905.8601


In [68]:
model.load_state_dict(torch.load("model_baseline.pt"))
model.eval

<bound method Module.eval of Model(
  (gru): GRU(43, 16, num_layers=2, batch_first=True, dropout=0.5)
  (linear): Linear(in_features=16, out_features=28, bias=True)
)>

In [69]:
test_loss = 0.0

for in_seq, target_seq in tqdm(test_loader):
    out_seq, _ = model(in_seq)
    loss = criterion(out_seq, target_seq)
    test_loss += loss.item()

test_loss /= len(test_loader)
print(f"Test loss: {test_loss:.4f}")

100%|██████████████████████████████████████████████████████████████████████████████| 3340/3340 [00:36<00:00, 91.03it/s]

Test loss: 952.3952


In [74]:
model.load_state_dict(torch.load("model_baseline_dropped_cols.pt"))
model.eval

<bound method Module.eval of Model(
  (gru): GRU(28, 16, num_layers=2, batch_first=True, dropout=0.5)
  (linear): Linear(in_features=16, out_features=28, bias=True)
)>

In [75]:
test_loss = 0.0

for in_seq, target_seq in tqdm(test_loader):
    out_seq, _ = model(in_seq)
    loss = criterion(out_seq, target_seq)
    test_loss += loss.item()

test_loss /= len(test_loader)
print(f"Test loss: {test_loss:.4f}")

100%|██████████████████████████████████████████████████████████████████████████████| 3340/3340 [00:40<00:00, 82.70it/s]

Test loss: 1070.2398


## Longer sequence

In [24]:
dropout_prob   = 0.5
embedding_size = 32
epoch_num      = 10
hidden_size    = 16
layer_num      = 2
learning_rate  = 1e-3
seq_size       = 50 #25 frames is equal to 1 second of video, maybe use 50?
pred_window    = 25
shift_size     = 25

In [32]:
class SeqDataset(Dataset):
    def __init__(self, device, dataframe, seq_size, pred_window, shift_size):
        super(SeqDataset, self).__init__()
        self.device = device
        self.seq_size = seq_size  # Input sequence length
        self.shift_size = 25  # Shift for the start of each new sequence
        self.pred_window = pred_window  # Additional frames for prediction
        self.dataframe = dataframe
        self.target_data = dataframe.filter(regex='x|y')

    def __len__(self):
        # The total number of sequences is adjusted for shifting the sequence start by self.shift_size
        return (len(self.dataframe) - self.seq_size - self.pred_window) // self.shift_size + 1

    def __getitem__(self, idx):
        # Calculate the actual starting index for the sequence based on the shift size
        start_idx = idx * self.shift_size
        end_idx = start_idx + self.seq_size
        target_end_idx = end_idx + self.pred_window

        # Ensure we don't exceed the bounds of the dataframe
        if target_end_idx > len(self.dataframe):
            target_end_idx = len(self.dataframe)
            end_idx = target_end_idx - self.pred_window

        in_seq = torch.tensor(self.dataframe.iloc[start_idx:end_idx].values,
                              dtype=torch.float, device=self.device)
        target_seq = torch.tensor(self.target_data.iloc[start_idx+self.pred_window:target_end_idx].values,
                                  dtype=torch.float, device=self.device)
        return in_seq, target_seq

In [33]:
# Split the data into train, validation, and test using the full video sizes (so that videos are not split into different sets)
train_data, test_data = train_test_split(data, test_size= 11250*int(0.2*98), shuffle=False)
n_train_vids = len(train_data)/11250
train_data, val_data = train_test_split(train_data, test_size = 11250 * int(0.1 * n_train_vids), shuffle=False)

# Create SeqDataset instances for the train, validation, and test sets
train_dataset = SeqDataset(device, train_data, seq_size, pred_window, shift_size)
val_dataset = SeqDataset(device, val_data, seq_size, pred_window, shift_size)
test_dataset = SeqDataset(device, test_data, seq_size, pred_window, shift_size)

# Create DataLoader instances for batching
batch_size = 64  # Adjust to your desired batch size
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, drop_last=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, drop_last=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, drop_last=True)

In [34]:
class Model(nn.Module):
    def __init__(self, dropout_prob, hidden_size, layer_num, input_size, output_size):
        super(Model, self).__init__()
#         self.embedding = nn.Embedding(vocab_size, embedding_size)
        self.gru       = nn.GRU(input_size, hidden_size, layer_num, batch_first=True, dropout=dropout_prob)
        self.linear    = nn.Linear(hidden_size, output_size)

    def forward(self, in_sequence, hidden_state=None):
#         embedding_seq            = self.embedding(in_sequence)
        hidden_seq, hidden_state = self.gru(in_sequence, hidden_state)
        out_seq                  = self.linear(hidden_seq)
        return out_seq, hidden_state
    
    def draw(self, in_sequence, logit_temp=1.0):
        out_seq, _  = self(in_sequence)
        prob_dist   = torch.softmax(out_seq[0, -1] / logit_temp, 0)
        rand_sample = torch.multinomial(prob_dist, 1).item()                   
        return rand_sample

In [35]:
input_size = next(iter(train_loader))[0].size(-1)
print("Input size:", input_size)
output_size = next(iter(train_loader))[1].size(-1)
print("Output size:", output_size)
model = Model(dropout_prob, hidden_size, layer_num, input_size, output_size).to(device)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), learning_rate)
print(model)

Input size: 43
Output size: 28
Model(
  (gru): GRU(43, 16, num_layers=2, batch_first=True, dropout=0.5)
  (linear): Linear(in_features=16, out_features=28, bias=True)
)


In [37]:
min_loss = float("inf")

for epoch in range(epoch_num):
    
    model.train()
    train_loss = 0.0
    
    i = 0
    for in_seq, target_seq in tqdm(train_loader, desc=f"Epoch {epoch}/{epoch_num}"):
        
        out_seq, _  = model(in_seq)
        loss        = criterion(out_seq, target_seq)
        train_loss += loss.item()
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        i += 1

    train_loss       /= len(train_loader)
    # train_perplexity  = np.exp(train_loss)
    print(f"Train loss: {train_loss:.4f}")

    model.eval()
    val_loss = 0.0

    with torch.no_grad():
        for in_seq, target_seq in val_loader:
            out_seq, _  = model(in_seq)
            loss        = criterion(out_seq, target_seq)
            val_loss   += loss.item()

    val_loss       /= len(val_loader)
    # val_perplexity  = np.exp(val_loss)
    print(f"Val loss: {val_loss:.4f}")

    if val_loss < min_loss:
        min_loss = val_loss
        torch.save(model.state_dict(), "C:/Users/Kieran/Documents/Master Thesis Data/Models/model_pred_25.pt")

Epoch 0/10: 100%|████████████████████████████████████████████████████████████████████| 506/506 [00:06<00:00, 73.52it/s]


Train loss: 14392.6587
Val loss: 13985.0300


Epoch 1/10: 100%|████████████████████████████████████████████████████████████████████| 506/506 [00:06<00:00, 78.47it/s]


Train loss: 13924.6838
Val loss: 13575.0200


Epoch 2/10: 100%|████████████████████████████████████████████████████████████████████| 506/506 [00:06<00:00, 80.14it/s]


...

Train loss: 11722.7404
Val loss: 11489.4610


Epoch 8/10: 100%|████████████████████████████████████████████████████████████████████| 506/506 [00:07<00:00, 69.89it/s]


Train loss: 11397.5210
Val loss: 11181.2699


Epoch 9/10: 100%|████████████████████████████████████████████████████████████████████| 506/506 [00:06<00:00, 81.33it/s]


Train loss: 11082.7356
Val loss: 10884.0909


In [38]:
# model.load_state_dict(torch.load("C:/Users/Kieran/Documents/Master Thesis Data/Models/model_pred_25.pt"))
model.eval

<bound method Module.eval of Model(
  (gru): GRU(43, 16, num_layers=2, batch_first=True, dropout=0.5)
  (linear): Linear(in_features=16, out_features=28, bias=True)
)>

In [39]:
test_loss = 0.0

for in_seq, target_seq in tqdm(test_loader):
    out_seq, _ = model(in_seq)
    loss = criterion(out_seq, target_seq)
    test_loss += loss.item()

test_loss /= len(test_loader)
print(f"Test loss: {test_loss:.4f}")

100%|███████████████████████████████████████████████████████████████████████████████| 133/133 [00:01<00:00, 102.39it/s]

Test loss: 11223.6390
